In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from time import sleep
from processor.processor import Processor as p
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [18]:
market.connect()
start = market.retrieve_max_date("crypto")
# start = datetime(2012,1,1).strftime("%Y-%m-%d")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")

2023-06-27 2023-06-26


In [20]:
crypto["ticker"] = "BTC"
market.connect()
market.drop("crypto")
market.store("crypto",crypto)
market.disconnect()

In [3]:
analysis = []
market.connect()
periods = ["year","quarter","month","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("stocks",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        ep = test.tail(1)["adjClose"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|████████████████████████████████████████████████████████████████████████| 490/490 [00:33<00:00, 14.69it/s]


In [4]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector,GICS Sub-Industry
25,0.516546,0.270872,0.060112,0.034743,0.042618,AMZN,Amazon,Consumer Discretionary,Internet & Direct Marketing Retail
189,0.321495,0.017873,0.071978,0.005526,0.031394,FDX,FedEx,Industrials,Air Freight & Logistics
148,-0.300062,-0.184931,0.064006,0.043821,0.030949,DG,Dollar General,Consumer Discretionary,General Merchandise Stores
334,0.117922,0.042391,0.077778,0.041766,0.030697,NRG,NRG Energy,Utilities,Independent Power Producers & Energy Traders
217,0.213332,0.114476,0.099041,0.031896,0.025855,HCA,HCA Healthcare,Health Care,Health Care Facilities
265,0.245294,0.171408,0.032816,0.006309,0.023991,KLAC,KLA Corporation,Information Technology,Semiconductor Equipment
270,0.501411,0.188606,0.000016,0.009100,0.023225,LRCX,Lam Research,Information Technology,Semiconductor Equipment
298,0.030197,0.044549,0.025151,0.031661,0.023030,MRK,Merck & Co.,Health Care,Pharmaceuticals
181,0.224110,0.128958,0.087878,0.003835,0.022093,EXPE,Expedia Group,Consumer Discretionary,Internet & Direct Marketing Retail
313,0.178247,0.114035,0.005559,0.022790,0.021914,MNST,Monster Beverage,Consumer Staples,Soft Drinks


In [5]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Industrials,0.106339,0.045933,0.075872,-0.000209,-0.006373
Information Technology,0.185738,0.038089,0.023628,-0.014525,-0.000406
Consumer Discretionary,0.130274,0.024394,0.074962,-0.004675,-0.003569
Health Care,0.020261,0.010760,0.041164,0.001484,0.003932
Communication Services,0.104013,0.004053,0.032547,-0.009945,-0.002603
Consumer Staples,0.018306,-0.002757,0.019519,0.005561,0.004263
Financials,-0.070865,-0.008529,0.029189,-0.014800,-0.012512
Real Estate,-0.026747,-0.010274,0.023110,-0.021067,-0.014659
Utilities,-0.038704,-0.014474,0.033842,-0.000726,-0.007227


In [6]:
## cloud db update
start = datetime.now() - timedelta(days=800)
market.connect()
stocks = market.retrieve("stocks")
tyields = market.retrieve("tyields")
bench = market.retrieve("spy")
market.disconnect()
stocks = p.column_date_processing(stocks)
stocks = stocks[stocks["date"]>=start]
market.cloud_connect()
market.drop("stocks")
market.drop("tyields")
market.drop("spy")
market.store("tyields",tyields)
market.store("spy",bench)
market.store("stocks",stocks)
market.create_index("stocks","ticker")
market.disconnect()